# Code to create training dataset for the vision model based on the previously collected data from the hand_signs_data_collection code.
(usefule link: https://saturncloud.io/blog/how-to-write-text-in-jupyter-ipython-notebook/)

In [1]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

DATA_DIR = 'data'
TRANSLATED_DATA_DIR = 'data_translated'

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.6, max_num_hands = 1)

In [3]:
data, labels = [], []
for dir_ in os.listdir(DATA_DIR):
    path = os.path.join(DATA_DIR, dir_)
    translated_data_path = os.path.join(TRANSLATED_DATA_DIR, dir_)
    if not os.path.exists(translated_data_path): os.makedirs(translated_data_path)
        
    for img_path in os.listdir(path):
        data_aux, x_, y_ = [], [], []

        img = cv2.imread(os.path.join(path, img_path))
        # Flip image horizontally to correctly identify left hand being used in the collected images
        #img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        # Only use the images with left hand for dataset generation => only 21 points for landmarks
        if results.multi_hand_landmarks and results.multi_handedness[0].classification[0].label == 'Left':
            for hand_landmarks in results.multi_hand_landmarks:
                # Write images with overlapped hand landmarks on the original data collected.
                mp_drawing.draw_landmarks(
                    img,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,  
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
                cv2.imwrite(os.path.join(translated_data_path, img_path), img)
                
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
    
                    x_.append(x)
                    y_.append(y)

            # To reduce variability for different positions of the hand on the screen, each data point subtracts the lowest x and y position per frame.
            # Essentially reducing the positions of all landmarks to be start at the bottom left conner of the image.
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append([x - min(x_), y - min(y_)])

            if data_aux:
                data.append(data_aux)
                labels.append(dir_)

In [4]:
# Store the data locally to be accessed by another code.
f = open('data.p', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()